In [10]:
from profile_alingment import *
from pair_alingment_HMM import *
from numpy import set_printoptions
set_printoptions(precision=2)
%precision 3

'%.3f'

# Task 4 - profile alingment

In [12]:
q = {"A": 0.25, "C": 0.25, "G": 0.25, "T": 0.25}
eM = [
    {"A": None, "C": None, "G": None, "T": None},
    {"A": 1/11, "C": 4/11, "G": 5/11, "T": 1/11},
    {"A": 2/10, "C": 1/10, "G": 1/10, "T": 6/10},
    {"A": 6/11, "C": 3/11, "G": 1/11, "T": 1/11},
]
eI = [
    {"A": 1/4, "C": 1/4, "G": 1/4, "T": 1/4},
    {"A": 1/4, "C": 1/4, "G": 1/4, "T": 1/4},
    {"A": 1/6, "C": 2/6, "G": 2/6, "T": 1/6},
    {"A": 1/4, "C": 1/4, "G": 1/4, "T": 1/4},
]
A = [
    {"MM": 8/10, "MD": 1/10, "MI": 1/10, "II": 1/3, "ID":1/3, "IM":1/3, "DD":None, "DI":None,"DM":None},
    {"MM": 7/10, "MD": 2/10, "MI": 1/10, "II": 1/3, "ID":1/3, "IM":1/3, "DD":1/3, "DI":1/3,"DM":1/3},
    {"MM": 5/9, "MD": 1/9, "MI": 3/9, "II": 1/5, "ID":1/5, "IM":3/5, "DD":1/4, "DI":1/4,"DM":2/4},
    {"MM": 8/9, "MD": None, "MI": 1/9, "II": 1/2, "ID":None, "IM":1/2, "DD":None, "DI":1/2,"DM":1/2},
]

seq = 'GTCCA' 

aligner = ViterbiProfileAligner(trans_matrix=A, gen_insert=eI, gen_match=eM, q=q, seq='-' + seq) # add slack 1st letter to seq
aligner.fit()
print(f"Alingment for {seq}: {aligner.alingment}")
print(f"Score: {aligner.score}")
print(f"States: {aligner.states}")
print("Dynamic prog table:")
for row in aligner.res:
    print(row)

alinger_forward = ForwardProfileAligner(trans_matrix=A, gen_insert=eI, gen_match=eM, q=q, seq='-' + seq)
alinger_forward.fit()
print(f"Probability of generation:{alinger_forward.score}")

Alingment for GTCCA: GTCCA
Score: 0.3370078971533515
States: ['M1', 'M2', 'I2', 'I2', 'M3']
Dynamic prog table:
[[0, -inf, -inf], [-inf, -2.3025850929940455, -inf], [-inf, -3.401197381662155, -inf], [-inf, -4.499809670330265, -inf], [-inf, -5.598421958998375, -inf], [-inf, -6.697034247666485, -inf]]
[[-inf, -inf, -2.3025850929940455], [0.3746934494414107, -3.401197381662155, -3.401197381662155], [-4.412798293340635, -1.9278916435526348, -4.499809670330265], [-4.125116220888854, -3.0265039322207445, -5.598421958998375], [-5.223728509556964, -4.125116220888854, -6.697034247666485], [-7.708635159344965, -5.223728509556964, -7.795646536334595]]
[[-inf, -inf, -3.401197381662155], [-4.31748811353631, -4.499809670330265, -1.2347444629926896], [0.8934872428565781, -3.026503932220745, -3.0265039322207445], [-3.9427946640948996, 0.08255702664024916, -4.125116220888854], [-5.041406952763009, -1.2391988133420704, -5.223728509556964], [-5.446872060871174, -3.2541018338843353, -6.322340798225074]]
[

# Task 5 - pairwise alingment with HMM

In [13]:
delta = 0.2
tau = 0.1
eps = 0.1
q = {"A": 0.25, "C": 0.25, "G": 0.25, "T": 0.25}
p = dict()
for l1 in ["A", "C", "G", 'T']:
    for l2 in ["A", "C", "G", 'T']:
        val = 0
        if l1 == l2:
            val = 0.5
        elif l1 in ["C", "T"] and l2 in ["C", "T"]:
            val = 0.05
        elif l1 in ["A", "G"] and l2 in ["A", "G"]:
            val = 0.05
        elif l1 in ["A", "T"] and l2 in ["A", "T"]:
            val = 0.3
        elif l1 in ["G", "C"] and l2 in ["G", "C"]:
            val = 0.3
        elif l1 in ["G", "T"] and l2 in ["G", "T"]: 
            val = 0.15
        elif l1 in ["A", "C"] and l2 in ["A", "C"]:
            val = 0.15              
        
        p[f"{l1}{l2}"] = val


seq1 = "TTAGTC"
seq2 = "TAG"

pw_alinger = ViterbiPairwiseAligner(p=p,q=q,delta=delta, tau=tau, eps=eps,s1='-' + seq1,s2='-' + seq2)
pw_alinger.fit()
print(f"Alingment of {seq1} to {seq2}:")
print(pw_alinger.alingment[0])
print(pw_alinger.alingment[1])
print("\nDynamic prog table")
for row in pw_alinger.res:
    print(row)

pw_alinger_forward = ForwardPairwiseAligner(p=p,q=q,delta=delta, tau=tau, eps=eps,s1='-' + seq1,s2='-' + seq2)
pw_alinger_forward.fit()
prob = pw_alinger_forward.score
print(f"Probability of generation:{prob}")

Alingment of TTAGTC to TAG:
TTAGTC
T-AG--

Dynamic prog table
[[1, 0, 0], [0, 0.05, 0], [0, 0.0012500000000000002, 0], [0, 3.125000000000001e-05, 0], [0, 7.812500000000002e-07, 0], [0, 1.9531250000000008e-08, 0], [0, 4.882812500000002e-10, 0]]
[[0, 0, 0.05], [0.25, 0.0, 0.0], [0.020000000000000004, 0.0125, 0.0], [0.0003000000000000001, 0.0010000000000000002, 0.0], [3.750000000000001e-06, 2.5000000000000008e-05, 0.0], [3.1250000000000013e-07, 6.250000000000003e-07, 0.0], [7.812500000000004e-10, 1.5625000000000006e-08, 0.0]]
[[0, 0, 0.0012500000000000002], [0.012000000000000002, 0.0, 0.0125], [0.0375, 0.0006000000000000002, 0.0010000000000000002], [0.005000000000000001, 0.001875, 1.5000000000000005e-05], [4.000000000000002e-05, 0.00025000000000000006, 1.8750000000000006e-07], [6.000000000000003e-06, 6.250000000000002e-06, 1.5625000000000006e-08], [7.500000000000002e-08, 3.0000000000000015e-07, 3.906250000000002e-11]]
[[0, 0, 3.125000000000001e-05], [0.00015000000000000004, 0.0, 0.0006000